# Basketball Question

### Importing Data / Packages

In [11]:
import pandas as pd
import numpy as np

In [12]:
#importing text documents
df_Event_Codes = pd.read_csv("Event_Codes.txt", sep = '\t')
df_Game_Lineup = pd.read_csv("Game_Lineup.txt", sep = '\t')

#sorting play by play as suggested in pdf
df_Play_by_Play = pd.read_csv("Play_by_Play.txt", sep = '\t').sort_values(['Game_id',
                                                                           'Period', 
                                                                           'PC_Time',
                                                                           'WC_Time',
                                                                           'Event_Num'],
                                                                          ascending = [0, 1, 0, 1, 1])

### Trying to Get Indicator variable for if player A is in

In [13]:
#creating merge of two dataframes

#gets correct team_id by merging play by play with game lineup. looking only at period 0 to extract info regardless if starter
df_merged = pd.merge(df_Play_by_Play.drop('Team_id', axis = 1),
                     df_Game_Lineup.loc[df_Game_Lineup.Period == 0][['Game_id', 'Person_id', 'Period', 'Team_id']],
                     left_on = ['Game_id', 'Person1'],
                     right_on = ['Game_id', 'Person_id']).drop('Period_y', axis = 1)

#merging to get information of whether player started specific period
df_merged = pd.merge(df_merged,
                     df_Game_Lineup[['Game_id', 'Person_id', 'Period']], 
                     how = 'left',
                     left_on = ['Game_id', 'Person1', 'Period_x'],
                     right_on = ['Game_id', 'Person_id', 'Period'])

#resorting and resetting index
df_merged.sort_values(['Game_id','Period_x', 'PC_Time', 'WC_Time', 'Event_Num'],
                                                                          ascending = [0, 1, 0, 1, 1], inplace = True)

df_merged.reset_index(inplace = True, drop = True)
               

In [4]:
#iterates through df_merged to find location of next non-freethrow event
def findNonFreethrow(i, player_index):
    num = 0
    while (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 3 or 
           df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 8 or 
          (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 6 and 
          df_merged.iloc[player_index[i + num]]['Action_Type'] in
          [11, 12, 13, 16, 18, 19, 21, 25, 30])) :
        num += 1
        
        if i + num >= len(player_index):
            return -1
        
    return num


#iterates through and checks if subsitituion occurs after free throw
def checkIfNotFreeThrow(i, player_index):
    num = 0
    while (df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 8 or 
          df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 6):
            num += 1
            if i + num >= len(player_index):
                return True
    
    if df_merged.iloc[player_index[i + num]]['Event_Msg_Type'] == 3:
        return False
    else:
        return True

In [5]:
#iterating through all players that were listed as starters or were subbed in
#creates column with name of player and puts 1 if player should be credited for points
df_answer = pd.DataFrame(columns = ['Game_ID', 'Player_ID', 'OffRtg', 'DefRtg'])

for player in df_Game_Lineup.Person_id.unique():
    
    #setting indicator value to 0 for all entries to start
    df_merged[player] = 0
    
    #defining boolean statement to select games of interest (games in which player played in)
    games = df_merged.Game_id.isin(df_Game_Lineup.loc[df_Game_Lineup.Person_id == player].Game_id.unique())

    #setting indicator value to 1 for those who start periods
    period_start_index = df_merged.loc[games].drop_duplicates(['Game_id', 'Period_x']).index 

    #groups by period finds players that started their period and puts a 1 in the first entry for those who started their period
    start_of_period = ((df_merged.loc[games].groupby(['Game_id', 'Period'])
                    .apply(lambda x: (x['Person1'] == player).any())).astype(int))
    

    
    #if player doesnt play, no need to set anything
    if len(start_of_period) > 0: 
        start_of_period = start_of_period.reset_index().sort_values(['Game_id', 'Period'],
                            ascending = [0, 1]).reset_index().set_index(period_start_index)
        
        df_merged.loc[period_start_index, player] = start_of_period[0]
    
    
    #setting continual indexes/indicators of being in game
    player_index = df_merged.loc[games].index
    freethrow_indicator = np.ones(len(player_index))
    
    df_merged[player + '_poss'] = np.NaN
    df_merged[player + '_poss'][0] = 0
    
    #going through player index
    for i in range(len(player_index) - 1):
        
        #defining these once
        freethrow_check = checkIfNotFreeThrow(i+1, player_index)
        freethrow_index = findNonFreethrow(i+1, player_index)    
        
    
        df_merged[player].iloc[player_index[i+1]] = (#current value (if already 1, should stay same)
                                    df_merged[player].iloc[player_index[i+1]] +
                        
                                    #no change in value if game or quarter ends:
                                    int((df_merged['Period_x'].iloc[player_index[i+1]] == 
                                         df_merged['Period_x'].iloc[player_index[i]])
                                    & (df_merged['Game_id'].iloc[player_index[i+1]] ==
                                       df_merged['Game_id'].iloc[player_index[i]])) *
                                    
                                    #checking if substitution out of player occured during freethrows:
                                    freethrow_indicator[i] *
                                    
                                    #previous value
                                     (df_merged[player].iloc[player_index[i]]
                        
                                      #checking if play before or after was freethrow
                                      + int(freethrow_check) * 
                                      
                                    #subtract 1 if player has been subbed out
                                      +((- int((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and 
                                        (df_merged['Person1'].iloc[player_index[i+1]] == player))
                          
                                    #add 1 if player has been subbed in
                                       + (int((df_merged['Event_Msg_Type'].iloc[player_index[i]] == 8) and 
                                        (df_merged['Person2'].iloc[player_index[i]] == player)))))))
        
                  
        
        #subbing players out after freethrow
        check_sub_out = ((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and
            #checking if substitution is surounded by freethrows
                (not freethrow_check) &
            #checking if player is being subbed out
                (df_merged['Person1'].iloc[player_index[i+1]] == player) and
            #making sure freethrow wasnt at very end of lastm   game
                freethrow_index != -1 &
            #making sure period is same period
                (df_merged['Period_x'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Period_x'].iloc[player_index[i+1]]) and 
            #making sure game is same game
                (df_merged['Game_id'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Game_id'].iloc[player_index[i+1]]))
    
    
        #checking if substitution
        if (check_sub_out):
            freethrow_indicator[freethrow_index + i] = 0
        
        #subbing players in after freethrow        
        check_sub_in = ((df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8) and
         #checking if substitution is surounded by freethrows
                (not freethrow_check) and
            #checking if player is being subbed out
                (df_merged['Person2'].iloc[player_index[i+1]] == player) and 
            #making sure freethrow wasnt at very end of last game
                findNonFreethrow(i+1, player_index) != -1 and
            #making sure period is same period
                (df_merged['Period_x'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Period_x'].iloc[player_index[i+1]]) and 
            #making sure game is same game
                (df_merged['Game_id'].iloc[player_index[freethrow_index + i]]
                 == df_merged['Game_id'].iloc[player_index[i+1]]))
        
        
        #technical freethrows
        num = 0
        if (df_merged['Event_Msg_Type'].iloc[player_index[i+1]] == 8):
            while ((df_merged.iloc[player_index[i + 1 + num]]['Event_Msg_Type'] == 3 or 
                   df_merged.iloc[player_index[i + 1 + num]]['Event_Msg_Type'] == 8 or 
                  (df_merged.iloc[player_index[i + 1 + num]]['Event_Msg_Type'] == 6 and 
                  df_merged.iloc[player_index[i + 1 + num]]['Action_Type'] in
                    [11, 12, 13, 16, 18, 19, 21, 25, 30])) and
                  (df_merged['Period_x'].iloc[player_index[i + 1 + num]] == 
                                df_merged['Period_x'].iloc[player_index[i + 1]])):
                
                    num = num + 1
                    if (df_merged.iloc[player_index[i + 1 + num]]['Event_Msg_Type'] == 3 and 
                          df_merged.iloc[player_index[i + 1 + num]]['Action_Type'] in
                            [16, 21, 22]):
                        df_merged.iloc[player_index[i + 1 + num]][player] = 1
                
                                
        
        
         #checking if substitution
        if (check_sub_in):

                df_merged[player].iloc[player_index[freethrow_index + i+1]] = 1
                
        
    
        #counting posessions
        check_missed_shot = (((df_merged['Event_Msg_Type'].iloc[player_index[i]] == 2) or 
                             (df_merged['Event_Msg_Type'].iloc[player_index[i]] == 3)) and
            (df_merged['Event_Msg_Type'].iloc[player_index[i + 1]] == 4) and
            (df_merged['Team_id'].iloc[player_index[i]] != df_merged['Team_id'].iloc[player_index[i+1]]))
        
            
        
        
        
        check_made_final_free_throw = (df_merged['Event_Msg_Type'].iloc[player_index[i]] == 3 and
                                         df_merged['Option1'].iloc[player_index[i]] == 1 and
                                      df_merged['Action_Type'].iloc[player_index[i]] in [29, 26, 22, 19, 15, 12])
        
        check_turnover = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 5
        
        check_end_quarter = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 13
        
        check_made_field_goal = df_merged['Event_Msg_Type'].iloc[player_index[i]] == 1
        
        check_player_in = df_merged[player].iloc[player_index[i]] == 1
        
        if (check_player_in and (check_made_field_goal or check_made_final_free_throw or 
            check_missed_shot or check_turnover or check_end_quarter)):    
            df_merged[player + '_poss'].iloc[player_index[i]] = max(df_merged[player + '_poss'].dropna()) + 1
        
        
        
        

    df_merged[player + '_poss'].fillna(method = 'ffill', inplace = True)
    
    

    team = df_Game_Lineup.loc[df_Game_Lineup.Person_id == player].Team_id.unique()[0]
    
    for game in df_merged.loc[df_merged.Person2 == player].Game_id.unique():
    
        df_merged_temp = df_merged.loc[df_merged.Game_id == game]
    
        total_offense = (sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 1].loc[df_merged_temp['Team_id'] == team].Option1) +   
                sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 3].loc[df_merged_temp['Team_id'] == team].Option1 == 1))
    
        
        total_defense = (sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 1].loc[df_merged_temp['Team_id'] != team].Option1) +   
                sum(df_merged_temp.loc[df_merged_temp[player] == 1].loc[df_merged_temp['Event_Msg_Type'] == 3].loc[df_merged_temp['Team_id'] != team].Option1 == 1))

    
        try:
            offensive_rating = total_offense / (max(df_merged_temp[player + '_poss']) - min(df_merged_temp[player + '_poss'])) * 100
        except:
            offensive_rating = np.NaN
        try:
            defensive_rating = total_defense / (max(df_merged_temp[player + '_poss']) - min(df_merged_temp[player + '_poss']))  * 100
        except:
            defensive_rating = np.NaN

        df_answer = df_answer.append([pd.DataFrame([[game, player, offensive_rating, defensive_rating]], columns = df_answer.columns)])
    

C:\Users\Tom Bliss\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Tom Bliss\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


KeyboardInterrupt: 

In [ ]:
df_merged.to_csv('df_merged_3.csv')

In [ ]:
df_answer.to_csv('final2.csv')

# Testing

In [76]:
ratings = pd.read_csv('final.csv')
features = pd.read_csv('df_merged_2.csv')

test = pd.merge(features, 
        df_Event_Codes[['Event_Msg_Type', 'Action_Type', 'Event_Msg_Type_Description', 'Action_Type_Description']],
        on=['Event_Msg_Type', 'Action_Type'])

test.Event_Msg_Type_Description = test.Event_Msg_Type_Description.str.rstrip()

test.sort_values(['Game_id','Period_x', 'PC_Time', 'WC_Time', 'Event_Num'],
                                                                          ascending = [0, 1, 0, 1, 1], inplace = True)
test.reset_index(inplace = True, drop = True)
test = test[['Game_id','Event_Num','Person1','Person2','Period','Option1','Event_Msg_Type_Description','Action_Type_Description']]
test.head()


,Game_id,Event_Num,Person1,Person2,Period,Option1,Event_Msg_Type_Description,Action_Type_Description
0,ff42c1cba89fd3f3a7a6711a11d5576d,4,8d2127290c94bd41b82a2938734bc750,99104de2626f67c1fa2ce70504970c3f,1.0,0,Jump Ball,
1,ff42c1cba89fd3f3a7a6711a11d5576d,7,c5dd5b2e3b975f0849d9b74e74125cb9,0370a0d090da0d0edc6319f120187e0e,1.0,2,Missed Shot,Pullup Jump shot
2,ff42c1cba89fd3f3a7a6711a11d5576d,8,99104de2626f67c1fa2ce70504970c3f,0370a0d090da0d0edc6319f120187e0e,1.0,0,Rebound,Unknown
3,ff42c1cba89fd3f3a7a6711a11d5576d,9,616281dee946056b071699476fdee9ec,d81d912f81fa43178f423aa89a713e96,1.0,3,Made Shot,Jump Shot
4,ff42c1cba89fd3f3a7a6711a11d5576d,11,42e0d7167f04a4ff958c6442da0e6851,0370a0d090da0d0edc6319f120187e0e,1.0,2,Missed Shot,Jump Shot


In [77]:
game_id = 'ff42c1cba89fd3f3a7a6711a11d5576d'
person_id = 'ae53f8ba6761b64a174051da817785bc'

In [78]:
df_Game_Lineup[(df_Game_Lineup.Game_id == game_id) & (df_Game_Lineup.Person_id == person_id)]

,Game_id,Period,Person_id,Team_id,status
5710,ff42c1cba89fd3f3a7a6711a11d5576d,0,ae53f8ba6761b64a174051da817785bc,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5750,ff42c1cba89fd3f3a7a6711a11d5576d,2,ae53f8ba6761b64a174051da817785bc,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A


In [79]:
df_Game_Lineup[((df_Game_Lineup.Game_id == game_id) & (df_Game_Lineup.Period == 1))]

,Game_id,Period,Person_id,Team_id,status
5740,ff42c1cba89fd3f3a7a6711a11d5576d,1,8d2127290c94bd41b82a2938734bc750,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5741,ff42c1cba89fd3f3a7a6711a11d5576d,1,48ec4e6c52f418d5ca4ef510ba473ea0,45ba8fc87f55b1191c50c400dc7ed11c,A
5742,ff42c1cba89fd3f3a7a6711a11d5576d,1,616281dee946056b071699476fdee9ec,45ba8fc87f55b1191c50c400dc7ed11c,A
5743,ff42c1cba89fd3f3a7a6711a11d5576d,1,e814950408915f43de2b079dce7c21c5,45ba8fc87f55b1191c50c400dc7ed11c,A
5744,ff42c1cba89fd3f3a7a6711a11d5576d,1,766802a8fda500d7945950de7398c9c6,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5745,ff42c1cba89fd3f3a7a6711a11d5576d,1,99104de2626f67c1fa2ce70504970c3f,45ba8fc87f55b1191c50c400dc7ed11c,A
5746,ff42c1cba89fd3f3a7a6711a11d5576d,1,42e0d7167f04a4ff958c6442da0e6851,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5747,ff42c1cba89fd3f3a7a6711a11d5576d,1,c5dd5b2e3b975f0849d9b74e74125cb9,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5748,ff42c1cba89fd3f3a7a6711a11d5576d,1,c10b49616a2f4a23607dc1a8be4fde9f,01be0ad4af7aeb1f6d2cc2b6b6d6d811,A
5749,ff42c1cba89fd3f3a7a6711a11d5576d,1,d81d912f81fa43178f423aa89a713e96,45ba8fc87f55b1191c50c400dc7ed11c,A


In [84]:
game = test[test.Game_id == game_id]

# substitutions for that player
# sub out
game[(game.Person1 == person_id) & (game.Event_Msg_Type_Description == 'Substitution')]
# sub in 
game[(game.Person2 == person_id) & (game.Event_Msg_Type_Description == 'Substitution')]

# what quarter the player started

,Game_id,Event_Num,Person1,Person2,Period,Option1,Event_Msg_Type_Description,Action_Type_Description
92,ff42c1cba89fd3f3a7a6711a11d5576d,134,f4a5ca938177c407a9dab5412e39498f,ae53f8ba6761b64a174051da817785bc,NaN,0,Substitution,
227,ff42c1cba89fd3f3a7a6711a11d5576d,342,42e0d7167f04a4ff958c6442da0e6851,ae53f8ba6761b64a174051da817785bc,NaN,0,Substitution,


In [82]:
game[(game.Person2 == person_id)].Event_Msg_Type_Description[92]

'Substitution'